In [1]:
include("../src.jl")

romberg

# Метод Ромберга

*Метод Ромберга* (*Romberg's method*) позволяет быстро интегрировать с заданной точностью. Метод основывается на применении экстраполяции Ричардсона к формуле трапеций при удвоении числа узлов.

## Экстраполяция Ричардсона

Частный случай этого метода мы рассматривали при выводе {ref}`Формулы Симпсона <sec:quadrature_simpson>`.

{cite}`SamarskiyGulin1989` Рассмотрим вычисление интеграла $I_h$ на равномерной сетке с шагом $h$. На каждом интервале отрезка применяется одна и та же квадратурная формула. Для некоторых формул (например, для формулы трапеций) удаётся получить вид погрешности $I - I_h$. Предположим, что это разложение выглядит так

```{math}
:label: richardson_int

I_h = I + a_1 h^{\alpha_1} + a_2 h^{\alpha_2} + ... + a_m h^{\alpha_m} + O(h^{\alpha_{m+1}}).
```

Здесь $0 < \alpha_1 < \alpha_2 < ... < \alpha_m$ -- степени разложения, а $a_i$ его ненулевые коэффициенты.

Теперь рассмотрим {eq}`richardson_int` на последовательности сеток с шагами $h_0 = h$, $h_1 = qh$, $h_2 = q^2 h$, ..., $h_k = q^k h$, $k=0,...,m$, $0 < q < 1$. Тогда, согласно {eq}`richardson_int`, запишем

```{math}
:label: richardson_decompose

I_{h_k} = I + a_1 h_k^{\alpha_1} + a_2 h_k^{\alpha_2} + ... + a_m h_k^{\alpha_m} + O(h_k^{\alpha_{m+1}}),\quad k=0,...,m.
```

Введём обозначение $I^{(1)}_{h_k} = I_{h_k}$ и запишем для двух сеток

```{math}
I^{(1)}_{h_{k-1}} = I + a_1 h_{k-1}^{\alpha_1} + O(h^{\alpha_2}_{k-1})\\
I^{(1)}_{h_k} = I + a_1 h_k^{\alpha_1} + O(h^{\alpha_2}_{k}).
```

Скомбинируем выражения так, чтобы слагаемое при $a_1$ занулить, получим

```{math}
:label: richardson_step

I = I^{(2)}_{h_{k-1}} + O(h^{\alpha_2}_{k-1}),
```

где

```{math}
I^{(2)}_{h_{k-1}} = \frac{I^{(1)}_{h_k} - q^{\alpha_1}I^{(1)}_{h_{k-1}}}{1 - q^{\alpha_1}}.
```

```{index} экстраполяция Ричардсона
```
Таким образом, мы выполнили один шаг **экстраполяции Ричардсона**. Важно отметить, что согласно {eq}`richardson_step` $I^{(2)}$ приближает $I$ лучше, чем $I^{(1)}$, у которых ошибка порядка $h^{\alpha_1}$.

Процесс экстраполяции можно продолжить, тогда в общем случае получим рекурретное выражение

```{math}
:label: richardson_general

\begin{align}
I_{h_{k-1}}^{(j+1)} = \frac{I^{(j)}_{h_k} - q^{\alpha_j}I^{(j)}_{h_{k-1}}}{1 - q^{\alpha_j}},\\
j = 1,..., m, \quad k = 1, 2, ..., m - j + 1, \\
I_{h_k}^{(1)} = I_{h_k},\quad k =0,1,...,m.
\end{align}
```

Точность приближения для {eq}`richardson_general` составляет

```{math}
I = I_{h_k}^{(j)} + O(h_k^{\alpha_j}).
```

Важно отметить структуру формулы {eq}`richardson_general`: $(j+1)$-ое приближение на сетке с шагом $h_{k-1}$ можно получить по $(j)$-приближениям на сетках с шагами $h_k$ и $h_{k-1}$.

```
\ j  1    2    3   
k --------------
0   01 → 02 → 03
       ↗    ↗
1   11 → 12
       ↗
2   21
```

```{index} метод; Ромберга
```
```{index} Ромберга метод
```
Теперь рассмотрим частный случай экстраполяции Ричардсона, **метод Ромберга**.

Положим в основу экстраполяции формулу трапеций и будем мельчить сетку вдвое ($q=0.5$). Для составной формулы трапеций разложение {eq}`richardson_decompose` содержит только *чётные степени*, т.е. $\alpha_j = 2 j$. Тогда, из {eq}`richardson_general` получим

```{math}
:label: romberg_method

I_{h_{k-1}}^{(j+1)} = \frac{4^j I^{(j)}_{h_k} - I^{(j)}_{h_{k-1}}}{4^j - 1}.
```

Прежде чем переходить к реализации, рассмотрим особенность формулы трапеции при измельчении сетки, это позволит избежать перевычислений в {eq}`romberg_method`.

## Удвоение числа узлов

Рассмотрим равномерную сетку $a = x_0 < x_1 < x_2 < x_3 < x_4 = b$ из $n=4$ интервалов с шагом $h_4$ и сетку из $n=2$ интервалов $a = x_0 < x_2 < x_4 =b$ с шагом $2h_4$.

%%% todo: сюда бы картинку

На крупной сетке формула трапеций даёт

```{math}
:label: doubling_tf2

T_f(n=2) = 2h_4 \Big[0.5 \big(f(x_0) + f(x_4)\big) + f(x_2)\Big].
```

А на сетке вдвое мельче

```{math}
:label: doubling_tf4

T_f(n=4) = h_4 \Big[ 0.5 \big(f(x_0) + f(x_4)\big) + f(x_1) + f(x_3) \Big].
```

Сравнивая {eq}`doubling_tf2` и {eq}`doubling_tf4`, обнаруживаем их связь

```{math}
T_f(n=4) = 0.5 T_f(n=2) + h_4 \big(f(x_1) + f(x_3)\big).
```

Таким образом, мы в частном случае получили, что вычислив интеграл на крупной сетке, мы можем вычислить его на сетке вдвое мельче, используя только значения в *новых узлах* ($x_1$, $x_3$).

Это верно и в общем случае

```{math}
:label: trapezoid_2x

T_f(2n) = 0.5 T_f(n) + h_{2n} \sum_{i=1}^n f(x_{2i-1}), \quad h_{2n} = \frac{b-a}{2n},\quad x_{2i-1} = a + h_{2n} (2i-1).
```

Данная формула позволяет вычислить интеграл с заданной точностью. Под точностью здесь понимается разница между новой аппроксимацией и старой

```{math}
:label: doubling_tol

|T_f(2n) - T_f(n)| < \text{tol}.
```

Например, при $\text{tol} = 10^{-6}$ мы можем добиться точности до 6 знаков после запятой. Конечно, ошибки округления и обусловленность задачи никуда не делись.

Также отметим, что выражение {eq}`doubling_tol` является абсолютной, а не относительной точностью.

Реализация может быть такой

(function:trapezoid_tol)=
:::{proof:function} trapezoid_tol

**Формула трапеций с контролем точности**

```julia
"""
Вычисляет интеграл ∫`f`dx на [`a`, `b`] с точностью `atol` по формуле трапеций,
удваивая число разбиений интервала, но не более `maxstep` раз.
Возвращает значение интеграла.
"""
function trapezoid_tol(f, a, b; atol=1e-3, maxstep::Integer=100)
    nc, hc = 1, b - a
    Tc = hc * (f(a) + f(b)) / 2
    for step in 1:maxstep
        Tp, np = Tc, nc
        hc /= 2
        nc *= 2
        Tc = Tp / 2 + hc * sum(f, (a + hc*(2i-1) for i in 1:np))
        abs(Tc - Tp) < atol && return Tc
    end
    error("Точность не удовлетворена.")
end
```
:::

## Реализация

```{margin}
Можно обойтись и двумя векторами.
```
Итак, хранить $I_{h_{k-1}}^{(j)}$ будем в элементе матрицы `I[k, j]`.

Первый элемент `I[1, 1]` заполним вручную: он соответствует формуле трапеций на сетке из одного интервала (2 узла). Затем, пока не достигнем необходимой точности, будем делать следующее

1. Измельчим сетку вдвое и подсчитаем интеграл по формуле трапеций на ней {eq}`trapezoid_2x`, так получим из `I[k, 1]` элемент `I[k+1, 1]`;
2. Вычислим по доступной побочной диагонали слева направо все элементы от `I[k, 2]` вплоть до `I[1, k+1]` по формуле перехода {eq}`romberg_method`;
3. Будем продолжать 1 и 2 до тех пор, пока приближения `I[1, k+1]` и `I[2, k]` станут неотличимы в заданной точности `atol`, или пока не исчерпаем количество измельчений `maxstep`.

**Пример заполнения матрицы `I`**. Допустим, максимальное измельчение `maxstep = 4`, т.е. от $h=b-a$ до $h = (b-a) / 2^4$. Ниже приведена схема вычислений и обозначения

- ⋅: элемент матрицы, который не пригодится в вычислениях;
- □: элемент, который, возможно, предстоит определить;
- ○ и ●: уже вычисленные элементы, причём ● сравниваются между собой в конце итерации;
- стрелка вниз показывает переход по умельчению сетки (шаг 1);
- остальные стрелки показывают переход по формуле (шаг 2).

```{margin}
Этот алгоритм является типичной задачей динамического программирования.
```
```
     1 итерация             2 итерация           3 итерация             4 итерация

○ → ●   □   □   □      ○   ○ → ●   □   □      ○   ○   ○ → ●   □      ○   ○   ○   ○ → ●
↓ ↗                          ↗                          ↗                          ↗
●   □   □   □   ⋅      ○ → ●   □   □   ⋅      ○   ○ → ●   □   ⋅      ○   ○   ○ → ●   ⋅
                       ↓ ↗                          ↗                          ↗
□   □   □   ⋅   ⋅      ○   □   □   ⋅   ⋅      ○ → ○   □   ⋅   ⋅      ○   ○ → ○   ⋅   ⋅
                                              ↓ ↗                          ↗
□   □   ⋅   ⋅   ⋅      □   □   ⋅   ⋅   ⋅      ○   □   ⋅   ⋅   ⋅      ○ → ○   ⋅   ⋅   ⋅
                                                                     ↓ ↗
□   ⋅   ⋅   ⋅   ⋅      □   ⋅   ⋅   ⋅   ⋅      □   ⋅   ⋅   ⋅   ⋅      ○   ⋅   ⋅   ⋅   ⋅
```

(function:romberg)=
:::{proof:function} romberg

**Метод Ромберга с контролем точности**

```julia
"""
Вычисляет интеграл ∫`f`dx на отрезке [`a`, `b`] методом Ромберга.
Разбивает отрезок пополам не более `maxstep` раз.
Возвращает значение интеграла, если приближения отличаются не более чем на `atol`.
"""
function romberg(f, a, b; atol=1e-6, maxstep::Integer=100)
    maxstep = max(2, maxstep)
    I = Matrix{Float64}(undef, maxstep+1, maxstep+1)
    I[1, 1] = (b - a) * (f(a) + f(b)) / 2
    for i in 2:maxstep+1
        let hc = (b - a) / 2^(i-1), np = 2^(i-2)
            I[i, 1] = I[i-1, 1] / 2 + hc * sum(f, (a + hc * (2i-1) for i in 1:np))
        end
        for k in i-1:-1:1
            I[k, i-k+1] = (2^i*I[k+1, i-k] - I[k, i-k]) / (2^i - 1)
        end
        abs(I[1, i] - I[2, i-1]) < atol && return I[1, i]
    end
    error("Точность не удовлетворена.")
end
```
:::

:::{proof:demo}
:::

```{raw} html
<div class="demo">
```

Сравним время вычисления интеграла

```{math}
\int_0^3 x e^{\sin 2x}
```

нашими реализациями {ref}`trapezoid_tol <function:trapezoid_tol>` и {ref}`romberg <function:romberg>`.

Вычисление по формуле трапеций с заданной точностью занимает

In [2]:
foo(x) = x * exp(sin(2x))
a, b = 0, 3
atol = 1e-6
Tint = @btime trapezoid_tol($foo, a, b; atol=$atol) seconds=1
Rint = @btime romberg($foo, a, b; atol=$atol) seconds=1
(Tint, Rint)

  589.263 μs (29491 allocations: 524.59 KiB)


  8.480 μs (6 allocations: 79.89 KiB)


(4.115935482633098, 4.115980256204295)

```{raw} html
</div>
```